In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models

import skorch
from skorch import NeuralNetClassifier

from pathlib import Path
import os
import sys
import time
import copy

import pandas as pd
import matplotlib.pylab as plt
import numpy as np

# Local modules
from cub_tools.train import train_model
from cub_tools.visualize import imshow, visualize_model
from cub_tools.utils import unpickle, save_pickle
from cub_tools.transforms import makeDefaultTransforms

In [2]:
# Script runtime options
model_names = ['resnet152', 'resnext101_32x8d', 'inception_v3', 'googlenet']
data_parallel = {'resnet152' : False,
                 'inceptionv4' : True,
                 'resnext101_64x4d' : True, 
                 'pnasnet5large' : True, 
                 'googlenet' : False,
                 'inception_v3' : False, 
                 'resnext101_32x8d' : False}
data_root_dir = '../data'
model_root_dir = '../models'
stages = ['train', 'test']


# Paths setup
data_dir = os.path.join(data_root_dir,'images')

In [3]:
# Get data transforms
data_transforms = makeDefaultTransforms()

In [4]:
# Setup data loaders with augmentation transforms
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                  for x in stages}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16,
                                             shuffle=True, num_workers=4)
              for x in stages}
dataset_sizes = {x: len(image_datasets[x]) for x in stages}
class_names = image_datasets[stages[0]].classes

In [5]:
# Setup the device to run the computations
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device::', device)

Device:: cuda:0


In [6]:
models = {}

for model_name in model_names:
    
    print('[INFO] Loading model {}'.format(model_name))
    
    # Paths
    output_dir = os.path.join(model_root_dir,'classification/{}'.format(model_name))
    model_file = os.path.join(output_dir, 'caltech_birds_{}_full.pth'.format(model_name))
    
    # Load the best model from file
    models[model_name] = torch.load(model_file)
    if data_parallel[model_name]:
        models[model_name] = torch.nn.DataParallel(model[model_name])
    models[model_name] = models[model_name].to(device)

[INFO] Loading model resnet152
[INFO] Loading model resnext101_32x8d
[INFO] Loading model inception_v3
[INFO] Loading model googlenet


In [ ]:
net = {}
for model_name in model_names:
    net[model_name] = NeuralNetClassifier(models[model_name])
    net[model_name].initialize()

## Ensemble Methods

Different approaches for ensembling

### Stacking ensemble bespoke implementation

Following the tutorial here: https://machinelearningmastery.com/stacking-ensemble-for-deep-learning-neural-networks/

In [24]:
from numpy import dstack
from sklearn.linear_model import LogisticRegression

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

# TODO: refactor so that it accepts a dataloader instance and iterates over the whole dataset.
#       Add as an outer loop and then run each model on an inner loop, so that they use the same images.
#       Remember to also keep the labels for later use and return them
def stacked_dataset(models, inputs):
    stackX = None
    for model_name, model in models.items():
        # make prediction
        yhat = model.predict_proba(inputs)
        for i in np.arange(0, yhat.shape[0], 1):
            yhat[i, ::] = softmax(yhat[i,::])
        # stack predictions into [rows, members, probabilities]
        if stackX is None:
            stackX = yhat
        else:
            stackX = dstack((stackX, yhat))
        print('{}...'.format(model_name), end='')
    # flatten predictions to [rows, members x probabilities]
    stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
    return stackX

def stacked_dataset_from_dataloader(models, dataloader, device):
    stackX = None
    stacky = None
    print('[INFO] Starting StackX', end='')
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloader):
            while i < (len(dataloader)-1):
                temp_stack = None
                for model_name, model in models.items():
                    # make prediction
                    if isinstance(model, skorch.classifier.NeuralNetClassifier):
                        yhat = model.predict_proba(inputs)
                    else:
                        model.eval()
                        inputs = inputs.to(device)
                        yhat = model(inputs)
                        yhat = yhat.cpu().numpy()

                    # Convert score to probability
                    for i in np.arange(0, yhat.shape[0], 1):
                        yhat[i, ::] = softmax(yhat[i,::])

                    # stack predictions into [rows, members, probabilities]
                    if temp_stack is None:
                        temp_stack = yhat
                    else:
                        temp_stack = dstack((temp_stack, yhat))

                # flatten predictions to [rows, members x probabilities]
                temp_stack = temp_stack.reshape((temp_stack.shape[0], temp_stack.shape[1]*temp_stack.shape[2]))
                # stack the batch of model probabilities onto the bottom of the results table
                if stackX is None:
                    stackX = temp_stack
                else:
                    stackX = np.vstack((stackX, temp_stack))

                # stack the output truth labels to bottom of truth labels table
                if stacky is None:
                    stacky = labels.cpu().numpy()
                else:
                    stacky = np.vstack((stacky, labels.cpu().numpy()))
                    
                if i % 25 == 0:
                    print('{}..'.format(i), end='')
    
    print('Complete')
    return stackX, stacky

# fit a model based on the outputs from the ensemble members
def fit_stacked_model(models, dataloader=None, stackX=None, labels=None, meta_learner=LogisticRegression):
    # create dataset using ensemble
    if (stackX is None) or (labels is None):
        print('[INFO] Creating the meta learner inputs (probabilities from individual models) as none provided.')
        stackedX, labels = stacked_dataset_from_dataloader(models, dataloader)
    else:
        print('[INFO] Stacked input table and labels found, using these to train meta learner.')
    
    # fit standalone model
    print('[INFO] Training the meta learner...', end='')
    meta_model = meta_learner()
    meta_model.fit(stackedX, labels)
    print('Complete')
    return meta_model

In [25]:
# Generate the stacked dataset from the ensemble models for the training data
stackY, stackx = stacked_dataset_from_dataloader(models, dataloaders['train'], device)

[INFO] Starting StackX

KeyboardInterrupt: 

In [ ]:
meta_model = fit_stacked_model(models=models, stackX=stackX, labels=labels)

In [9]:
dataloaders['train'].batch_size

16

In [18]:
len(dataloaders['train'])

375

In [ ]:
for model_name, model in models.items():
    print(model)

In [ ]:
inputs, classes = next(iter(dataloaders['train']))

net = {}
output = {}
with torch.no_grad():
    for model_name in model_names:

        net[model_name] = NeuralNetClassifier(model[model_name])
        net[model_name].initialize()
        
        output[model_name] = net[model_name].predict(inputs)

for i_res, truth in enumerate(classes.numpy()):
    res_str = 'Truth: {:5}  Pred:'.format(truth)
    for model_name in model_names:
        res_str = res_str + ' {:5}'.format(output[model_name][i_res])
    print(res_str)

In [ ]:
from combo.models.classifier_comb import SimpleClassifierAggregator
#from sklearn.ensemble import VotingClassifier

In [ ]:
combo_model = SimpleClassifierAggregator(base_estimators=list(net.values()),
                                         method='majority_vote',
                                         pre_fitted=True)

In [ ]:
combo_model.predict(inputs)